In [1]:
%%capture 
!pip install langchain openai
!pip install -q -U faiss-cpu tiktoken


In [2]:
import os 
import getpass
os.environ["OPENAI_API_KEY"]= getpass.getpass("API KEY: ")


In [3]:

from langchain.document_loaders import WebBaseLoader

yolo_nas_loader=WebBaseLoader("https://deci.ai/blog/yolo-nas-object-detection-foundation-model/").load()

decicoder_loader= WebBaseLoader("https://deci.ai/blog/decicoder-efficient-and-accurate-code-generation-llm/#:~:text=DeciCoder's%20unmatched%20throughput%20and%20low,re%20obsessed%20with%20AI%20efficiency.").load()

yolo_newsletter_loader=WebBaseLoader("https://deeplearningdaily.substack.com/p/unleashing-the-power-of-yolo-nas").load()


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( chunk_size=500, chunk_overlap=50, length_function=len)

yolo_nas_chunks=text_splitter.transform_documents(yolo_nas_loader)
decicoder_chunks = text_splitter.transform_documents(decicoder_loader)
yolo_newsletter_chunks = text_splitter.transform_documents(yolo_newsletter_loader)


In [5]:

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store=LocalFileStore("./cache/")

In [7]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedder = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")


In [9]:
#store embeddings in vector store
vectorstore = FAISS.from_documents(yolo_nas_chunks,embedder)
vectorstore.add_documents(decicoder_chunks)
vectorstore.add_documents(yolo_newsletter_chunks)

#instantiate a retriever
retriever= vectorstore.as_retriever()

In [16]:
# from langchain import PromptTemplate
# from langchain.chains.question_answering import load_qa_chain


In [17]:
# prompt_template = """
#   Please answer the question in as much detail as possible based on the provided context.
#   Ensure to include all relevant details. If the answer is not available in the provided context,
#   kindly respond with "The answer is not available in the context." Please avoid providing incorrect answers.
# \n\n
#   Context:\n {context}?\n
#   Question: \n{question}\n

#   Answer:
# """

# prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])


In [19]:

# #Load the model
# from langchain_google_genai import ChatGoogleGenerativeAI
# model = ChatGoogleGenerativeAI(model="gemini-pro",
#                              temperature=0.3)

# chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

# question = input("Enter your question: ")
# docs = retriever.get_relevant_documents(question)

# response = chain(
#     {"input_documents":docs, "question": question}
#     , return_only_outputs=True)
# response


{'output_text': 'The answer is not available in the context.'}

retrievalQA

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [24]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

In [28]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
handler = StdOutCallbackHandler()

In [29]:
qa_with_sources_chain= RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)


In [30]:
response = qa_with_sources_chain({"query":"What does Neural Architecture Search have to do with how Deci creates its models?"})



> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
print(response['result'])

The provided context does not contain information about Neural Architecture Search and how Deci creates its models. Therefore, I cannot answer this question.
